# Synthetic Data Generation for Coca-Cola Marketing Mix
**A Step-by-Step Guided Case Study**

## 🥤 Introduction: The Coca-Cola Scenario
Welcome! In this class, you are a Business Analyst at Coca-Cola. You don't need to be a programmer to use this tool. Think of the code cells below as "Engines" that generate a year of sales data for us.


# Step 1. Configuration and Library Installation

Before starting, we need to load our tools. This section sets up the required Python libraries. `Faker` is essential for generating realistic, structured categorical IDs (like Campaign IDs and Market Codes), adding a layer of corporate data complexity.

Just click the "Play" button on the left of the cell below. You don't need to change anything here.

In [ ]:
# Just press play!

# Install Faker (if necessary)
!pip3 install pandas numpy faker seaborn matplotlib

# Import libraries
import pandas as pd
import numpy as np
from faker import Faker
import random
import seaborn as sns
import matplotlib.pyplot as plt

random.seed(42)


🤖 **Meet "Faker": Our Digital Reality Generator**

In the business world, we often face a problem: Real data is sensitive. Due to privacy laws (like GDPR) or corporate secrets, we cannot always use real customer names or internal codes for training or testing.

Faker is a Python library that acts like a "digital actor" factory. It allows us to generate thousands of realistic pieces of information in seconds. Here you have some interesting links:
- [Faker Documentation](https://faker.readthedocs.io/en/master/#)
- [Faker Example 1](https://www.datacamp.com/tutorial/creating-synthetic-data-with-python-faker-tutorial)
- [Faker Example 2](https://www.geeksforgeeks.org/python/python-faker-library/)

Business Note: In your future career, tools like Faker are essential for creating "Prototypes" or "Demos" of products before you have real customers.

In [ ]:
# Initialize Faker
fake = Faker('en_US')
print("✅ Our generator is ready to go!")

In [ ]:
# Run this cell multiple times to see different fake identities!
print(f"Fake Employee Name: {fake.name()}")
print(f"Fake Warehouse Address: {fake.address()}")
print(f"Fake Business Email: {fake.company_email()}")

Before we build our Coca-Cola dataset, let's learn how to generate data.

To use this tool, you need to understand its structure:

1. **The Generator (fake)**: This is the main tool.

2. **Providers**: These are like "departments" (e.g., the Address department, the Job department).

3. **Generator Properties**: These are the specific "items" you can get, like a name or a color.

**Key Rule**: Each of the generator properties (like `name`, `address`, and `email`) are called "fake". A faker generator has many of them, packaged in "providers".

🔍 **Exercise**: The Provider Hunt
Your task is to visit the [Faker Documentation](https://faker.readthedocs.io/en/master/providers.html) and find the right "Property" to fill the "Secret Agent Passport".

In [ ]:
# --- TODO: ---
# Replace the ???? with the correct property from the documentation

print("🕵️ SECRET AGENT PASSPORT")
print("-" * 30)

# 1. Use the 'Person' provider
print(f"FULL NAME: {fake.name()}")

# 2. Find the property for a full credit card number in the 'Credit Card' provider
print(f"SAFE HOUSE: {fake.????()}") 

# 3. Find the property for a job in the 'Job' provider
print(f"COVER JOB: {fake.????()}") 

# 4. Find the property for a company name in the 'Company' provider
print(f"FRONT COMPANY: {fake.????()}") 

print("-" * 30)

Now that we know how to generate synthetic data, let's return to our **Coca-Cola** use case.

This notebook contains the Python code to generate a synthetic dataset for a Marketing Mix Modeling (MMM) scenario, incorporating Time Series variables, regulatory impact (Legal), and segmentation (Computer Science/Marketing). The final dataset contains the columns that you can see below:

| Variable (Field)        | Data Type             | Description                                                                                                   | Key Discipline Focus              |
|-------------------------|-----------------------|---------------------------------------------------------------------------------------------------------------|-----------------------------------|
| Date                    | Date                  | Observation date (daily granularity over 365 days).                                                            | Computer Science (Time Series)    |
| Market_Code             | Categorical (String)  | Internal, synthetic market code for the region (e.g., MK-N01). Generated using Faker.                          | Computer Science / Marketing      |
| Region                  | Categorical           | Geographic segmentation: North, South, Central, East.                                                          | Marketing / Legal                 |
| Product_Flavor          | Categorical           | Specific product variant: Classic, Vanilla_Zero, Cherry_Zero, Mango_Zero.                                     | Marketing                         |
| Campaign_ID             | Categorical (String)  | Identifier for the campaign period (e.g., SPR-#### for Spring campaign). Generated using Faker.                | Marketing                         |
| Sugar_Tax_Regulation    | Binary (0/1)          | Regulatory flag: 1 if the region has a strict sugar tax or regulation; 0 otherwise. Critical for Legal Analysis.| Legal                             |
| Customer_Segment_ID     | Integer (1-4)         | Synthetic cluster ID assigned to the consumer base in that market/day.                                         | Computer Science (Clustering)     |
| Ad_Impressions          | Integer               | Daily volume of digital advertising impressions served in the market. Simulates campaign spikes.              | Marketing                         |
| Promo_Type              | Categorical           | Type of promotional activity active that day: None, Discount_10, BOGO.                                        | Marketing                         |
| Competitor_Price_Index  | Numeric (Float)       | Price of the main competitor relative to Coca-Cola (1.0 = price parity).                                      | Marketing                         |
| Avg_Social_Sentiment    | Numeric (Float)       | Average sentiment score (0 to 1) towards the campaign/brand on social media that day.                          | Computer Science / Marketing      |
| Hashtag_Mentions        | Integer               | Daily count of relevant hashtag mentions on social media platforms.                                            | Computer Science / Marketing      |
| Sales_Volume            | Integer               | TARGET VARIABLE: Total units of the specific Product_Flavor sold daily in the Region.                          | Computer Science / Marketing      |

# Step 2. Defining our regions

We define the core constants for the Marketing Mix Modeling (MMM) scenario. 

We operate in 4 regions. Some regions (South and East) have a Sugar Tax, meaning the government charges extra for sugary drinks. This will affect how much we sell.

Note the **`SUGAR_TAX_REGIONS`** array; this is a critical input that links the Legal and Computer Science domains, as it will be used to penalize the 'Classic' flavor's sales.

In [ ]:
# CONFIGURATION AREA
# Here we define the basic 'rules' of our market: regions and flavors
N_DAYS = 365 
REGIONS = ['North', 'South', 'Central', 'East']
FLAVORS = ['Classic', 'Vanilla_Zero', 'Cherry_Zero', 'Mango_Zero']

print(f"Market defined: {len(REGIONS)} Regions and {len(FLAVORS)} Flavors across {N_DAYS} days.")


In [ ]:
# --- TODO: ---
# Instructions: Edit the list below to define which regions have strict sugar regulations.
# Make sure the names match exactly: 'North', 'South', 'Central', or 'East'.
SUGAR_TAX_REGIONS = []

# Step 3. Creating the basic structure of data

Now we will generate a list of daily sales. We will use `Faker` to create realistic "Market Codes" for each region so our database looks professional. First we will create the basic structure of our data.

In [ ]:
N_ROWS = N_DAYS * len(REGIONS) * len(FLAVORS)
print(f"Total rows to generate (Day * Region * Flavor): {N_ROWS}\n")

# Create the base matrix (Date, Region, Flavor)
dates = pd.date_range(start='2024-01-01', periods=N_DAYS, freq='D')
base_df = pd.DataFrame([(d, r, f) for d in dates for r in REGIONS for f in FLAVORS],
                       columns=['Date', 'Region', 'Product_Flavor'])

print(base_df.head()) # Check the base structure

# Step 4. Marketing Impact
We use custom logic and `Faker` to generate the foundational columns, focusing on identifiers and the regulatory binary variable.

In a large company like Coca-Cola, marketing actions are organized by **Campaign IDs**. These are unique codes (like SPR-4921) used by the Finance and Marketing departments to track how much money was spent and earned during a specific period.

We have designed a "Smart Labeler" (a function) that looks at the date of every single sale and decides if it belongs to a special campaign:

- Spring Campaign (March & April): Labeled as SPR-####
- Summer Campaign (July & August): Labeled as SMR-####
- Regular Days: Labeled as BASE-0000

**The Magic** of bothify

To make it look real, we use a `Faker` property called `bothify`.

- It takes a pattern like SPR-####.
- It automatically replaces every # with a random number.
- This creates a unique, professional-looking ID for our simulation.

In [ ]:
# --- Faker Integration for Realism ---
def generate_campaign_ids(df):
    campaign_ids = []
    id_spring = fake.bothify(text='SPR-####')
    id_summer = fake.bothify(text='SMR-####')
    
    for _, row in df.iterrows():
        month = row['Date'].month
        if month in [3, 4]:
            campaign_ids.append(id_spring)
        elif month in [7, 8]:
            campaign_ids.append(id_summer)
        else:
            campaign_ids.append('BASE-0000')
    return campaign_ids

Now we run the code and save the results in a new column called **`Campaign_ID`**:

In [ ]:
base_df['Campaign_ID'] = generate_campaign_ids(base_df)

In professional databases, regions are rarely identified only by their names (like "North"). They usually have a Market Code for internal logistics, accounting, and supply chain tracking.

In this step, we are doing two things:

1. **Creating a Dictionary**: We build a "translation table" that says: "For the North region, the code will be MK-N[Random Numbers]".

2. **Mapping the Table**: We tell the computer to look at our big sales table and, wherever it sees "North", "South", etc., add its corresponding unique code.

How the code works:
* `r[0].upper()`: This tells Python: "Take the first letter of the Region and make it Capital" (N for North, S for South).

* `'MK-' + ... + '##'`: This creates a pattern or "template" like MK-N##.

* `fake.bothify`: As you learned before, this replaces the ## with random numbers (e.g., MK-N12).

* `.map()`: This is the command that performs the "translation" for all rows in our table at once.

In [ ]:
# --- MARKET CODE GENERATION ---
# This part creates the "Translation Dictionary"
# Format: { 'RegionName': 'MK-Letter##' }
market_code_map = {r: fake.bothify(text='MK-' + r[0].upper() + '##') for r in REGIONS}
base_df['Market_Code'] = base_df['Region'].map(market_code_map)

In data analysis, we often need to translate a complex reality into a simple binary format (1 or 0). This methodology will be used for variable **`Sugar_Tax_Regulation`**. This variable links the Legal domain to the data, as it will directly impact the target variable. Possible values will be:

* 1: Yes, this condition is true (The region has a Sugar Tax).

* 0: No, this condition is false (The region has no tax).

This allows the computer to perform mathematical operations later. For example, if we want to penalize sales by 20%, we can multiply by this variable.

In [ ]:
# --- LEGAL REGULATION FILTER ---

# This line creates our binary 'Yes/No' column (1 or 0)
# 'lambda r' is just a way to say: "For every region (r) in our list..."
base_df['Sugar_Tax_Regulation'] = base_df['Region'].apply(lambda r: 1 if r in SUGAR_TAX_REGIONS else 0)

Finally, in marketing, treating every customer exactly the same is a mistake. A teenager buying a single can of Cherry Zero at a gas station is not the same as a parent buying three 24-packs of Classic Coke at a supermarket.

Column **`Customer_Segment_ID`**, acts as a crucial bridge between simple data collection and advanced Artificial Intelligence; while we are currently assigning every sale to one of four groups (1 to 4) randomly, this field represents the "hooks" that marketing teams use to perform Clustering, allowing you in the future to replace these random numbers with real consumer profiles (such as "Budget Shoppers" or "Health-Conscious Teens"), to analyze if certain segments are more loyal to specific flavors or more sensitive to price changes and government regulations.

In [ ]:
# --- CUSTOMER SEGMENTATION ---
# We generate a random ID between 1 and 4 for every row (N_ROWS)
# These represent different 'types' of shoppers
base_df['Customer_Segment_ID'] = np.random.randint(1, 5, N_ROWS)

print(base_df.head())  # Check the updated structure with new columns

# Step 5. Generation of Marketing Mix and Digital Variables

In business, sales don't just happen by accident, they are driven by specific actions. This step simulates our Marketing Mix—the levers we pull to convince customers to buy a Coke.

The first variable we will use to model this will be **`Campaign_ID`**. This variable will measure the number of impressions a campaign has had among the public.


Why are we **"spiking"** the Ads?
We will implement the code to intentionally "spike" our Digital Ad Impressions during the Spring and Summer campaigns.

* Normal days: Between 1,000 and 5,000 views.

* Spring Campaign: Boosted up to 15,000 views.

* Summer Campaign: Boosted up to 20,000 views.

We do this because a good analyst needs to see "contrast" in the data. If we spent the same amount on ads every single day, we would never know if the ads actually work! By creating these peaks, we can later prove the Return on Investment (ROI) of our marketing budget.

In [ ]:
# --- MARKETING MIX GENERATION ---

# 1. We start with a baseline of random daily ad views
base_df['Ad_Impressions'] = np.random.randint(1000, 5000, N_ROWS)

# 2. We "Inject" the Campaign Spikes
peak_spring = base_df['Campaign_ID'].str.contains('SPR')
peak_summer = base_df['Campaign_ID'].str.contains('SMR')

base_df.loc[peak_spring, 'Ad_Impressions'] = np.random.randint(5000, 15000, peak_spring.sum())

# --- TODO: ---
# Replace the ???? with the correct code to inject summer campaign spikes
base_df.loc[peak_summer, 'Ad_Impressions'] = ????

Secondly, we define two critical business variables that directly influence a customer's decision at the point of sale:

* **`Promo_Type`**: We have three options: None (regular price), Discount_10 (10% off), and BOGO (Buy One Get One Free).
    * We use a probability distribution p=[0.7, 0.2, 0.1]. This means that in our simulation, 70% of the time there is no promo, 20% of the time we offer a discount, and only 10% of the time we run a BOGO campaign.

* **`Competitor_Price_Index`**: This is a benchmark.
    * A value of 1.0 means our price is identical to our competitors.

    * Values below 1.0 mean we are cheaper (more competitive).

    * Values above 1.0 mean we are more expensive (premium positioning).

    * We use np.random.uniform(0.8, 1.2) to simulate a realistic market where prices fluctuate daily within a +/- 20% range.

This two features allow the model to reflect a realistic environment where our sales are constantly pressured by how our prices compare to rivals like Pepsi.

In [ ]:
# --- PROMOTION AND PRICING STRATEGY ---

# 1. Define the types of promotions available
PROMOS = ['None', 'Discount_10', 'BOGO']

# 2. Assign promotions to each day based on our strategy:
# 70% None, 20% Discount, 10% BOGO
base_df['Promo_Type'] = np.random.choice(PROMOS, N_ROWS, p=[0.7, 0.2, 0.1])

# 3. Generate a Price Index compared to competitors (between 0.8 and 1.2)
# We round to 2 decimals for a cleaner "Price Tag"
base_df['Competitor_Price_Index'] = np.random.uniform(0.8, 1.2, N_ROWS).round(2)

# Final check
print("Promotion and Pricing variables successfully applied!")
# Let's see how they distributed
base_df['Promo_Type'].value_counts(normalize=True)

In modern marketing, we don't just look at our own ads, we look at what the internet says about us. We are going to generate two "Digital Signals":

* **`Avg_Social_Sentiment`**: This represents the mood of the internet.

    * Values close to 0.9 mean the brand is trending positively (everyone loves the new ad!).

    * Values close to 0.1 mean there is a public relations crisis.

    * We use a Normal Distribution (Bell Curve) because most days are just "normal" (around 0.6).

* **`Hashtag_Mentions`**: This represents the volume of conversation (How many times #CocaCola was used).
**Instructions:**
1. Research: Go to the [Official NumPy Documentation for Poisson](https://numpy.org/doc/stable/reference/random/generated/numpy.random.poisson.html).

2. Find the Parameter: Look for the parameter that represents the "interval average" (often called lambda or lam).

3. Implement: In the code below, replace the ???? with the correct function and parameters.

    * The average number of mentions should be 20.

    * The size of the data should be N_ROWS (which we defined at the beginning).

In [ ]:
# --- DIGITAL INPUTS ---

# 1. Social Sentiment (The "Mood" of the internet)
# Most days it's around 0.6 (Normal distribution)
base_df['Avg_Social_Sentiment'] = np.random.normal(loc=0.6, scale=0.15, size=N_ROWS).clip(0.1, 0.9)

# TODO: Change the 'lam' (lambda) value to increase or decrease daily mentions
base_df['Hashtag_Mentions'] = np.random.????(lam=??, size=N_ROWS)

In [ ]:
# --- TEST YOUR CODE ---
if 'Hashtag_Mentions' in base_df.columns:
    print("✅ Great job! You've generated the digital signal.")
    print(f"Average mentions generated: {base_df['Hashtag_Mentions'].mean():.2f}")
    # Show the 'spikes' of virality
    base_df['Hashtag_Mentions'].plot(kind='hist', title='Distribution of Hashtag Mentions', bins=20)
else:
    print("❌ The column 'Hashtag_Mentions' is missing. Keep investigating!")

# Step 6. Generation of the Target Variable (`Sales_Volume`)

This is the most critical section. `Sales_Volume` is generated as a complex, non-linear function of all inputs.

The core dependencies include:
1.  **Seasonality:** Higher sales in summer months.
2.  **Marketing Mix:** Multiplicative effect from promotions (`BOGO` > `Discount`).
3.  **Ad Impressions:** Logarithmic (non-linear) additive effect, simulating diminishing returns.
4.  **Regulatory Impact (Legal):** A **25% penalty** is applied to `Classic` sales in regions where `Sugar_Tax_Regulation` = 1.

Before we calculate the impact of our marketing campaigns or the sugar tax, we need to establish a **Baseline**. This represents what Coca-Cola would sell on an average day, with no special events or regulations. In this initial phase of our sales modeling, we establish a Baseline by generating a "natural" sales volume using a Normal Distribution, which provides a realistic starting point centered around an average of 1,500 daily units.

In [ ]:
# --- BASELINE SALES GENERATION ---

# We generate the "natural" sales for every day in our dataset
base_sales = np.random.normal(loc=1500, scale=300, size=N_ROWS)
sales_volume = base_sales

print(f"✅ Baseline sales generated. {sales_volume.mean():.2f} average sales before seasonality.")

In the beverage industry, the weather is one of our most powerful "silent" sales agents. When the temperature rises, so does the thirst of our consumers. In analytics, we call this **Seasonality**.

To reflect this reality in our data, we apply a "Summer Factor":

* The Logic: The code identifies the months of June (6), July (7), and August (8).

* The Multiplier: We add a 40% boost (0.4 + 1.0 = 1.4) to the baseline sales during these months.

* The Result: If a normal day sells 1,500 units, a summer day will automatically jump to 2,100 units just because of the calendar.

In [ ]:
# --- APPLYING SEASONALITY ---

# 1. Identify rows that fall in June, July, or August
# This creates a 'multiplier' (1.4 for summer, 1.0 for the rest of the year)
summer_factor = base_df['Date'].dt.month.isin([6, 7, 8]) * 0.4 + 1.0

# 2. Multiply our baseline sales by this factor
sales_volume *= summer_factor
print(f"✅ Seasonality applied: Summer sales boost included. {sales_volume.mean():.2f} average sales after seasonality.")

Marketing isn't just one thing, it's a combination of "levers" we pull to increase demand. In this step, we simulate two of the most common ones:

1. **Promotion Power (Multiplicative)**: We use a "mapping" to scale sales based on the offer:

    * Discount_10: Boosts sales by 25% (multiplier of 1.25).

    * BOGO: Boosts sales by 55% (multiplier of 1.55).

    * None: Sales stay at 100% (multiplier of 1.0).

2. **The Ad Effect (Logarithmic)**: We add the influence of our Digital Ad Impressions.

    * We use a Logarithm (np.log) because of the "Law of Diminishing Returns."

    * The first 1,000 ads have a huge impact, but the difference between 1,000,000 and 1,001,000 ads is almost invisible. This prevents our simulation from creating "infinite sales" just by spending more money.

In [ ]:
# --- APPLYING PROMOTIONS & ADVERTISING ---

# 1. Map each promo type to its specific sales boost
promo_map = {'None': 1.0, 'Discount_10': 1.25, 'BOGO': 1.55}
sales_volume *= base_df['Promo_Type'].map(promo_map)

# 2. Add the impact of digital ads (using a log scale to be realistic)
sales_volume += np.log(base_df['Ad_Impressions'] + 1) * 35 

print("Marketing Mix applied! We can now see how promos and ads drive the volume.")

In business, your price is only "expensive" or "cheap" compared to your neighbor. This step introduces **Competitive Pressure**.

We use the **`Competitor_Price_Index`** to adjust our sales volume using an inverse relationship:

* If the Index is 1.2 (We are 20% more expensive): Our sales will suffer because customers might switch to a cheaper brand.

* If the Index is 0.8 (We are 20% cheaper): Our sales will increase as we "steal" market share.

The Formula Logic: We use (Index * 0.5 + 0.5) to smooth the impact.

* This ensures that even if our price is high, sales don't drop to zero instantly, reflecting real-world Brand Loyalty. People still buy Coca-Cola even if it's slightly more expensive than a generic brand, but they buy less of it.

In [ ]:
# --- COMPETITOR PRICE EFFECT ---

# We adjust the volume based on how our price compares to the market
# High Index (>1.0) = Lower Sales | Low Index (<1.0) = Higher Sales
sales_volume *= (base_df['Competitor_Price_Index'] * 0.5 + 0.5) 

print("Competitive effect applied. The market is now reacting to our pricing strategy.")


n the digital age, a brand's health isn't just measured by clicks, but by **how people feel**. This step adds the "Digital Halo" effect to our sales volume.

* **Quality over Quantity**: While Ad_Impressions measured how many people saw us, Avg_Social_Sentiment measures if they actually like us.

* **Direct Impact**: We multiply the sentiment score (from 0.1 to 0.9) by 150 units.

    * If the internet is "on fire" with positive comments (0.9), we add 135 extra units per day.

    * If there is a PR crisis (0.1), we only add 15 units.

This simulates a world where a good reputation acts as a sales accelerator, rewarding the brand when the community is happy.

In [ ]:
# --- DIGITAL SENTIMENT EFFECT ---

# A higher sentiment score directly adds more units to our daily volume
sales_volume += base_df['Avg_Social_Sentiment'] * 150

print("Digital reputation applied! The 'mood' of the internet is now impacting our bottom line.")


This is the most critical part of the simulation for the **Legal Department**. Here, we apply the "real-world" consequences of the sugar tax regulations we defined earlier. Notice how the law doesn't affect everyone equally: it creates winners and losers.

1. **The Penalty (Classic Coke)**: For our Classic flavor in regulated regions, we apply a 25% sales cut (* 0.75). This simulates how higher prices (due to the tax) or health warnings drive consumers away.

2. **The Opportunity (Zero Sugar)**: For products containing Zero in their name, we apply a 5% boost (* 1.05) in those same regions. This simulates "Substitution Bias"—consumers who want a soda but switch to the healthier, non-taxed option.

In [ ]:
# --- APPLYING LEGAL REGULATION IMPACT ---

# 1. Identify "Classic" sales in Taxed Regions (The Losers)
is_classic_and_taxed = (base_df['Product_Flavor'] == 'Classic') & (base_df['Sugar_Tax_Regulation'] == 1)
sales_volume[is_classic_and_taxed] *= 0.75 

# 2. Identify "Zero" sales in Taxed Regions (The Winners)
is_zero_and_taxed = (base_df['Product_Flavor'].str.contains('Zero')) & (base_df['Sugar_Tax_Regulation'] == 1)
sales_volume[is_zero_and_taxed] *= 1.05 

print("Legal impact applied! The regulation has reshaped our sales landscape.")

In the real world, even if you follow every formula, sales will never be exactly what you predicted. A truck might break down, a local festival might boost sales unexpectedly, or a fridge in a supermarket might stop working.

To make our data truly professional, we add two final touches:

1. **Random Noise**: We add a small amount of "randomness" (np.random.normal(0, 100)). This ensures that two days with the exact same ads, price, and weather still have slightly different sales. It's the "human factor."

2. **The Floor (Safety Logic)**: Since it is mathematically impossible to have "negative sales" (you can't sell -50 Cokes), we use np.maximum(0, ...) to ensure that the lowest possible value is zero.

In [ ]:
# --- FINAL REFINEMENT ---

# 1. Add final "Market Noise" (standard fluctuation)
sales_volume += np.random.normal(0, 100, N_ROWS)

# 2. Convert to integers (you can't sell half a bottle) 
# and ensure no negative values exist
base_df['Sales_Volume'] = np.maximum(0, sales_volume.astype(int))

print("Sales_Volume generated with injected correlations.")

# Step 7. Coherence Validation and Documentation

Before handing the data off, the CS team must validate that the injected logic works. The validation check below is essential for the **Legal** team, as it visually quantifies the economic effect of the `Sugar_Tax_Regulation`.

In [ ]:
# Reorder columns for presentation and saving
final_cols = ['Date', 'Market_Code', 'Region', 'Product_Flavor', 'Campaign_ID',
              'Sugar_Tax_Regulation', 'Customer_Segment_ID',
              'Ad_Impressions', 'Promo_Type', 'Competitor_Price_Index',
              'Avg_Social_Sentiment', 'Hashtag_Mentions', 'Sales_Volume']

synthetic_data = base_df[final_cols]

print("\n--- Final Dataframe (Head) ---")
display(synthetic_data.head(10))



In [ ]:
# --- COHERENCE VALIDATION (CS/Legal Task) ---
print("\n--- Regulatory Impact Validation (Average Sales) ---")
validation_check = synthetic_data.groupby(['Product_Flavor', 'Sugar_Tax_Regulation'])['Sales_Volume'].mean().unstack()
print("Average 'Classic' sales SHOULD be lower where Sugar_Tax_Regulation=1.")
print("Average 'Zero' sales SHOULD be higher where Sugar_Tax_Regulation=1.")
display(validation_check)

In [ ]:
# --- Save the dataset ---
#synthetic_data.to_csv('coca_cola_marketing_mix_synthetic_data_faker.csv', index=False)
print("\nDataset successfully saved as 'coca_cola_marketing_mix_synthetic_data_faker.csv'")

# 7. Basic Visualization Task

These visualizations serve as the final validation check, ensuring the data structure is robust and the correlations are visible before moving to advanced modeling.

In [ ]:
# Visualization Example 1: Validate Promotion Effect (Marketing Mix)
plt.figure(figsize=(8, 5))
sns.boxplot(x='Promo_Type', y='Sales_Volume', data=synthetic_data, order=PROMOS)
plt.title('Sales Volume vs. Promotion Type (Validation)')
plt.ylabel('Sales Volume (Units)')
plt.show()

In [ ]:
# Visualization Example 2: Validate Regulatory Effect (Legal)
plt.figure(figsize=(8, 5))
sns.barplot(x='Product_Flavor', y='Sales_Volume', hue='Sugar_Tax_Regulation', data=synthetic_data)
plt.title('Regulatory Impact on Sales by Flavor')
plt.show()

The **Seasonality Trend**: Does the "Summer Boost" actually show up in the calendar?

In [ ]:
# Visualization Example 3: Monthly Sales Trend
plt.figure(figsize=(12, 5))
base_df.set_index('Date').resample('M')['Sales_Volume'].sum().plot(marker='o', color='red', linewidth=2)
plt.title('Monthly Total Sales (Seasonality Validation)', fontsize=14)
plt.ylabel('Units Sold')
plt.xlabel('Month')
plt.show()

**Marketing ROI**: Are we getting diminishing returns on our ad spend?

In [ ]:
# Visualization Example 4: Sales vs. Ad Impressions (Diminishing Returns)
plt.figure(figsize=(10, 5))
sns.regplot(x='Ad_Impressions', y='Sales_Volume', data=base_df, 
            logx=True, scatter_kws={'alpha':0.2, 'color':'gray'}, line_kws={'color':'red'})
plt.title('Marketing Effectiveness: The Law of Diminishing Returns', fontsize=14)
plt.show()

The **Correlation Matrix**: Which variable has the strongest "hidden" link to our sales?

In [ ]:
# Visualization Example 5: The Business "Drivers"
plt.figure(figsize=(8, 6))
cols_of_interest = ['Sales_Volume', 'Ad_Impressions', 'Competitor_Price_Index', 'Avg_Social_Sentiment', 'Hashtag_Mentions']
correlation_matrix = base_df[cols_of_interest].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Business Variable Correlation Matrix', fontsize=14)
plt.show()